# Data Wrangling

## Dataset - WeRateDogs&trade; Twitter Archive

***By: Kartik Nanduri***<br>
**Date: 21st Nov, 2018.**

## Let's Gather

In [1]:
# importing all the necessary libraries
import os
import pandas as pd
import requests as req

***<span style="color: red">Important! uncomment the following files to run the book with out errors.</span>***

In [2]:
# resetting the folder structure.
#os.rename('dataset/twitter-archive-enhanced.csv', 'twitter-archive-enhanced.csv')
#import shutil
#shutil.rmtree('dataset')

***<span style="color: green">Important! once done, please recomment.</span>***

1. [x] **The file given at hand `twitter-archive-enhanced.csv`**

In [3]:
# all the requried files for this project are in the list files_list
files_list = ['twitter-archive-enhanced.csv', 'image-predictions.tsv', 'tweet_json.txt']

In [4]:
# reading the twitter archive file
archive = pd.read_csv(files_list[0])

# taking at random entries for the archive file
archive.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1668,682638830361513985,NaN,NaN,2015-12-31 19:05:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I have no words. Just a magnificent pup. 12/10...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/682638830...,12,10,None,None,None,None,None
524,809220051211603969,NaN,NaN,2016-12-15 02:14:29 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Kyro. He's a Stratocumulus Flop. Tongu...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/809220051...,12,10,Kyro,None,None,None,None


2. [x] **Fetching the data from url and saving it to local drive - `image-predictions.tsv`**

In [5]:
# reading the file from internet using the requests library
url = "https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5bf60c69_image-predictions-3/image-predictions-3.tsv"
res = req.get(url)

with open(files_list[1], mode = "wb") as op_file:
    op_file.write(res.content)

In [6]:
# checking if fetched the data right way
img_pre_test = pd.read_csv(files_list[1], delimiter = "\t", encoding = 'utf-8')
img_pre_test.sample(2)

# we did it the right way, Yay! it worked.

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1882,847157206088847362,https://pbs.twimg.com/media/C8G0_CMWsAAjjAY.jpg,2,Staffordshire_bullterrier,0.219609,True,American_Staffordshire_terrier,0.178671,True,pug,0.123271,True
1598,799297110730567681,https://pbs.twimg.com/media/CxeseRgUoAM_SQK.jpg,1,malamute,0.985028,True,Siberian_husky,0.005834,True,Eskimo_dog,0.005443,True


3. [x] **Getting data from Twitter&trade;**

In [7]:
# importing all the necessary libraries for accessing Twitter via API
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

In [8]:
# setting up all the necessary placeholders for API
consumer_key = 'xxx.xxx.xxx.xxx'
consumer_secret = 'xxx.xxx.xxx.xxx'
access_token = 'xxx.xxx.xxx.xxx'
access_secret = 'xxx.xxx.xxx.xxx'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth_handler = auth,
                 parser = tweepy.parsers.JSONParser(),
                 wait_on_rate_limit = True, 
                 wait_on_rate_limit_notify = True)

In [9]:
def fetch_and_save(ids, api_ins, one_id = None):
    '''
    This function will fetch data with associated id in ids list
    ids (List Object): a list all tweets
    api_ins (Tweepy Object): api object instance, will be used to query twitter for data
    one_id (int): use when you want to query only for one tweet
    failed_ids (List Object): a list will be retured so that, this fuction can be called once again on those ids
    '''
    new_file_name = ''; failed_ids = []; tweet_df = []
    
    # checking if file exists
    if os.path.exists(files_list[2]):
        temp = [s for s in os.listdir() if "tweet_json" in s]
        new_file_name = files_list[2].split('.')[0] + "_" + str(len(temp)) + ".txt"
    else:
        new_file_name = files_list[2]
    
    # querying a list of ids
    if one_id == None:
        with open(new_file_name, mode = 'w') as outfile:
            for one_id in ids:
                try:
                    content = api_ins.get_status(one_id, tweet_mode='extended')
                    json.dump(content, outfile)
                    outfile.write('\n')
                
                except Exception as e:
                    print("Error for: " + str(one_id) + " - " + str(e))
                    failed_ids.append(one_id)
    
    # querying a single id
    else:
        try:
            content = api_ins.get_status(one_id, include_entities = True)
            favorites = content['favorite_count']
            retweets = content['retweet_count']
            
            tweet_df.append({'tweet_id': int(one_id),
                        'favorites': int(favorites),
                        'retweets': int(retweets)})
            
            return tweet_df
                           
        except Exception as e:
            print("Error for: " + str(one_id) + " - " + str(e))
            failed_ids.append(one_id)

    return failed_ids

In [10]:
# passing the list of ids to the fuction fetch_and_save()
tweet_ids = archive['tweet_id'].tolist()
len(tweet_ids)

2356

In [11]:
# fetching data
# starting the timer
start = timer()

# querying
errors = fetch_and_save(tweet_ids, api)

# ending the timer
end = timer()

# calculating the runtime for fetch_and_save
print("That took about {} mins.".format(round((end - start)/60, 1)))

Error for: 888202515573088257 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 873697596434513921 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 872668790621863937 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 869988702071779329 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 866816280283807744 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 861769973181624320 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 845459076796616705 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 842892208864923648 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 837012587749474308 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 827228250799742977 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 802247111496568832 - [{'code': 144, 'message': 'No status found with 

Rate limit reached. Sleeping for: 499


Error for: 758740312047005698 - Failed to send request: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))
Error for: 754011816964026368 - [{'code': 144, 'message': 'No status found with that ID.'}]


Rate limit reached. Sleeping for: 500


Error for: 676957860086095872 - Failed to send request: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))
That took about 35.0 mins.


In [12]:
# lets save the failed ids into one master list
print("Total failed request are: {}. \n".format(len(errors)))

# ids that failed and the ones that passed
indi_fail = []; success = []

#for each failed id, lets try to fetch status individually.
for error in errors:
    temp = fetch_and_save(ids = None, api_ins = api, one_id = error)
    indi_fail.append(temp[0])

# removing empty elements from list
success = [x for x in indi_fail if not isinstance(x, (int))]
indi_fail = [x for x in indi_fail if isinstance(x, (int))]

# checking if there is change
print("\nWe were able to retrieve {} records, others failed.".format(len(errors) - len(indi_fail)))

Total failed request are: 16. 

Error for: 888202515573088257 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 873697596434513921 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 872668790621863937 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 869988702071779329 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 866816280283807744 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 861769973181624320 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 845459076796616705 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 842892208864923648 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 837012587749474308 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 827228250799742977 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 802247111496568832 - [{'code': 144, '

In [13]:
# printing the results of success
success

[{'tweet_id': 758740312047005698, 'favorites': 6078, 'retweets': 1742},
 {'tweet_id': 676957860086095872, 'favorites': 2302, 'retweets': 857}]

4. [x] **Okay, let's read `retweets` and `favourite count` from `tweet_json.txt`**

In [14]:
# reading tweet_json.txt
tweets = pd.read_json(files_list[2], lines = True, encoding = 'utf-8')

In [15]:
# let's select only the following columns retweet_count, favorite_count, id
tweets = tweets[['id', 'favorite_count', 'retweet_count']]
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 3 columns):
id                2340 non-null int64
favorite_count    2340 non-null int64
retweet_count     2340 non-null int64
dtypes: int64(3)
memory usage: 54.9 KB


In [16]:
# renaming the columns
tweets.rename(columns={'id': 'tweet_id',
                       'favorite_count': 'favorites',
                       'retweet_count': 'retweets'},
              inplace = True)

In [17]:
# concating the dataframes into one
success = pd.DataFrame(success, columns = ['tweet_id',
                                           'favorites',
                                           'retweets'])
tweet_master = pd.concat([tweets, success],
                         ignore_index = True,
                         sort = True,)
tweet_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2342 entries, 0 to 2341
Data columns (total 3 columns):
favorites    2342 non-null int64
retweets     2342 non-null int64
tweet_id     2342 non-null int64
dtypes: int64(3)
memory usage: 55.0 KB


In [18]:
# making a copy of the archive
archive_copy = archive.copy()

# marking the id null that we failed to retrieve in archive
for a_id in indi_fail:
    archive_copy.loc[archive_copy['tweet_id'] == a_id, ['tweet_id']] = 0

In [19]:
# a checking if we did it right
len(archive_copy[archive_copy['tweet_id'] == 0])

14

In [20]:
# appending the new file to our files list
files_list.append('archive_copy.csv')

In [21]:
# writing the contents of tweet_master to a file
import numpy as np
tweet_master['tweet_id'] = tweet_master['tweet_id'].astype(np.int64)
tweet_master.to_csv('tweet_master.csv', index = False)

# saving the updated version of our archived-enhanced.csv
archive_copy['tweet_id'] = archive_copy['tweet_id'].astype(np.int64)
archive_copy.to_csv(files_list[3], index = False)

5. [x] **Last thing to do is to tidy up our folder, let's get going.**

In [22]:
# moving all data files under one folder - dataset
# removing the temporary files, that acted as placeholders

# creating the folder
folder = 'dataset'
if not os.path.exists(folder):
    os.mkdir(folder)

# we know that our master datasets for this project are
# 1. twitter-archive-enhanced.csv
# 2. image-predictions.tsv
# 3. tweet_json.txt
# 4. tweet_master.csv
# let us move these files

# updating our files_list
files_list.append('tweet_master.csv')

# moving only required files
for file in files_list:
    if os.path.exists(file):
        os.rename(file, folder+'/'+file)
    
# lisitng the current directory
os.listdir()

# clean and neat, lets get with assessing and cleaning

['.git',
 '.ipynb_checkpoints',
 'dataset',
 'error.png',
 'New Text Document.txt',
 'README.md',
 'wrangle_act.ipynb']

In [23]:
# renaming files_list
for i in range(len(files_list)):
    files_list[i] = folder + '/'+ files_list[i]

## Summary - Gathering

- We know, that gathering is a the first step in wrangling.
- We were successful in gathering from three different sources with different techniques:
    - Data given at hand.
    - Fetch from flat file stored on a server.
    - From API.

- There a total of 14 missing data points, tried a different ways for retrieving them, using the API as well as `twurl` of the `Ruby` package, but they were not to be found, as stated below in the highlighted section.

***<span style="color: ##6c6cff">So let's start with assessing the data.</span>***

![error](error.png)

## Assessing

In [24]:
files_list

['dataset/twitter-archive-enhanced.csv',
 'dataset/image-predictions.tsv',
 'dataset/tweet_json.txt',
 'dataset/archive_copy.csv',
 'dataset/tweet_master.csv']

In [25]:
# let's load up dataset, and starting assessing them.
archive =  pd.read_csv(files_list[-2], encoding = 'utf-8')
img_pre = pd.read_csv(files_list[1], sep = '\t', encoding = 'utf-8')
tweet_master = pd.read_csv(files_list[-1], encoding =  'utf-8')

### Issues to sort!

In [26]:
# printing out archive - visual assessment
archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [27]:
# Programmatic Assessment 1 - Information
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [28]:
# Programmtic Assessment 2 - Describe
archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.378107e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,8.885894e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,0.000000e+00,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.776996e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.178159e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.986755e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [29]:
# checking for duplicates - tweet_ids
# these are the 0's for which the api failed to retrive data
sum(archive.tweet_id.duplicated())

13

In [30]:
# checkin if we have more than one class of dogs assigned to dog
# the following are the only combinations that are present in the dataset
cond_1 = (archive['doggo'] == 'doggo') & (archive['floofer'] == 'floofer')
cond_2 = (archive['doggo'] == 'doggo') & (archive['pupper'] == 'pupper')
cond_3 = (archive['doggo'] == 'doggo') & (archive['puppo'] == 'puppo')

# printing these entries
archive[cond_1 | cond_2 | cond_3][['tweet_id', 'text', 'doggo', 'floofer', 'pupper', 'puppo']]

,tweet_id,text,doggo,floofer,pupper,puppo
191,855851453814013952,Here's a puppo participating in the #ScienceMa...,doggo,None,None,puppo
200,854010172552949760,"At first I thought this was a shy doggo, but i...",doggo,floofer,None,None
460,817777686764523521,"This is Dido. She's playing the lead role in ""...",doggo,None,pupper,None
531,808106460588765185,Here we have Burke (pupper) and Dexter (doggo)...,doggo,None,pupper,None
565,802265048156610565,"Like doggo, like pupper version 2. Both 11/10 ...",doggo,None,pupper,None
575,801115127852503040,This is Bones. He's being haunted by another d...,doggo,None,pupper,None
705,785639753186217984,This is Pinot. He's a sophisticated doggo. You...,doggo,None,pupper,None
733,781308096455073793,"Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",doggo,None,pupper,None
778,775898661951791106,"RT @dog_rates: Like father (doggo), like son (...",doggo,None,pupper,None
822,770093767776997377,RT @dog_rates: This is just downright precious...,doggo,None,pupper,None


In [31]:
len(archive[cond_1 | cond_2 | cond_3])

14

1. **`twitter-archive-enhanced.csv`** table

***1 Content Issues:***

**1.1 Visual Assessment:**
- `rating_numerator` : has values such as 1, 3.. e.t.c - **Data Quality Dimension - `Consistency`**.
- `rating_denominator` : have values, less than 10, for example, the tweet_id - 666287406224695296 has the number 2 as its value - **Data Quality Dimension - `Consistency`**. 
- We see that, Articles - `a`, `an`, `the` have been used to name dogs, as well as words such as `such`, `quite` - **Data Quality Dimension - `Validity`**.
- There are instances where the names of dogs are in lowercases - **Data Quality Dimension - `Consistency`**.

**1.2 Programmatic Assessment:**
- `rating_numerator` : has a maximum value of 1766 - **Data Quality Dimension - `Consistency`**. 
- `rating_denominator` : has a maximum value of 170 - **Data Quality Dimension - `Consistency`**.
- All in all, this dataset appears to be clean, except for `expanded_url` - we have about 59 instances missing - **Data Quality Dimension - `Completeness`**.
- We can see that there are more than one class assigned to tweets, analyze and assign proper dog class so that melting is easy - **Data Quality Dimension - `Consistency`**.

***2 Structural Issues:***

**2.1 Visual Assessment:**
- we can see that, there are four classes of dogs `doggo`, `floofer`, `puppo`, `pupper`; these should a part of one unit - `dog_class` - **Data Quality Dimension - `Consistency`**.

**2.2 Programmatic Assessment:**
- `in_reply_to_status_id`, `retweeted_status_id`, `retweeted_status_user_id`, `in_reply_to_user_id` of type float64 must be converted into int - **Data Quality Dimension - `Validity`**.
- `timestamp`, `retweeted_status_timestamp` of type object must be converted into datatime - **Data Quality Dimension - `Validity`**.

In [32]:
# assessing img_predictions dataset
img_pre

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [33]:
# Programmatic Assessment - Information
img_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [34]:
# checking for duplicates
img_pre[img_pre['jpg_url'].duplicated(keep = False)].sort_values(by = 'jpg_url')[['tweet_id', 'jpg_url']]

,tweet_id,jpg_url
480,675354435921575936,https://pbs.twimg.com/ext_tw_video_thumb/67535...
1297,752309394570878976,https://pbs.twimg.com/ext_tw_video_thumb/67535...
1864,842892208864923648,https://pbs.twimg.com/ext_tw_video_thumb/80710...
1641,807106840509214720,https://pbs.twimg.com/ext_tw_video_thumb/80710...
1703,817181837579653120,https://pbs.twimg.com/ext_tw_video_thumb/81596...
1691,815966073409433600,https://pbs.twimg.com/ext_tw_video_thumb/81596...
1705,817423860136083457,https://pbs.twimg.com/ext_tw_video_thumb/81742...
1858,841833993020538882,https://pbs.twimg.com/ext_tw_video_thumb/81742...
1715,819004803107983360,https://pbs.twimg.com/media/C12whDoVEAALRxa.jpg
1718,819015337530290176,https://pbs.twimg.com/media/C12whDoVEAALRxa.jpg


2. **`image-predictions.tsv`** table

***1 Content Issues:***

**1.1 Visual Assessment:**
- We have few dog breeds that are represented in lowercase.

**1.2 Programmatic Assessment:**
- We have about 281 images on a whole, that are missing with respect to our `twitter-archive-enhanced.csv` file - **Data Quality Dimension - `Completeness`**.
- We can see that, we have about `66` duplicates **OR** a pair of tweets are pointing to same *`jpg_url`* - **Data Quality Dimension - `Accuracy`**.

***2 Structural Issues:***

**2.1 Visual Assessment:**
- None. 

**2.2 Programmatic Assessment:**
- None.

In [35]:
# assessing tweet_master dataset
tweet_master

,favorites,retweets,tweet_id
0,38181,8360,892420643555336193
1,32744,6176,892177421306343426
2,24652,4086,891815181378084864
3,41510,8496,891689557279858688
4,39692,9199,891327558926688256
5,19926,3059,891087950875897856
6,11650,2028,890971913173991426
7,64404,18545,890729181411237888
8,27377,4201,890609185150312448
9,31419,7260,890240255349198849


In [37]:
tweet_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2342 entries, 0 to 2341
Data columns (total 3 columns):
favorites    2342 non-null int64
retweets     2342 non-null int64
tweet_id     2342 non-null int64
dtypes: int64(3)
memory usage: 55.0 KB


3. **`tweet_master.txt`** table

***1 Content Issues:***

**1.1 Visual Assessment:**
- None.

**1.2 Programmatic Assessment:**
- We have about 14 missing records - **Data Quality Dimension - `Completeness`**.

***2 Structural Issues:***

**2.1 Visual Assessment:**
- None.

**2.2 Programmatic Assessment:**
- None.

## Summary - Assessing

- Completed the second step.
- The following are the insights:
    - from `twitter-archive-enhanced.csv` datset, the rating_numerator and denominator need to be fixed.
    - the dataset also represents row values as columns, which needs to be fixed.
    - the dataset also has structural issues such as wrong datatype assigned to a column.
    - from `images-preductions.tsv` dataset, there is consistency issue with naming dog breeds.
    - the dataset isn't complete when compared to `twitter-archive-enhanced.csv`, we have about 281 missing tweets.
    - Also we have `jpg_urls'` that are pointing to a pair of tweets.
    - `tweet_master.txt` dataset has about 14 missing records.
    - the dataset alone hold the information about retweets and favourites - bad form of schema normalization.

## Cleaning

#### Define
- Important!, before we get to cleaning, let's drop rows from image-predictions, that are false in dog_1,_2 and _3, as they are not related to our dataset.

#### Code

In [38]:
# only select those rows that are either true or false and not all false
img_pre = img_pre[~((img_pre.p1_dog == False) & (img_pre.p2_dog == False) & (img_pre.p3_dog == False))]

####  Test

In [39]:
# asserting the lenght to be 0
assert len(img_pre[(img_pre.p1_dog == False) & (img_pre.p2_dog == False) & (img_pre.p3_dog == False)]) == 0, "Check"

In [40]:
# the master dataset
master_set = archive.merge(img_pre, how = 'left', on = ['tweet_id'])
master_set = master_set.merge(tweet_master, how = 'left', on = ['tweet_id'])
files_list.append('dataset/master_set_raw.csv')
master_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 30 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
jpg_url                       17

In [42]:
# saving the file to local disk.
master_set.to_csv(files_list[-1], index = False)

In [43]:
# creating a copy of the master set
master_set = pd.read_csv(files_list[-1], encoding = 'utf-8')
master_copy = master_set.copy()

### Issues to Clean.

#### 1. Basic cleaning.

#### Define
- Assign proper class for the above 14 tweets before melting.
- Delete *retweets* with *any duplicates* and get rid of *tweets with **no** images*.
- Once done, drop the following columns:
    1. `retweeted_status_id`
    2. `retweeted_status_user_id`
    3. `retweeted_status_timestamp`
    4. `in_reply_to_status_id`
    5. `in_reply_to_user_id`
    
#### Code

In [44]:
# setting column width to -1
pd.set_option('display.max_colwidth', -1)
cond_1 = (master_copy['doggo'] == 'doggo') & (master_copy['floofer'] == 'floofer')
cond_2 = (master_copy['doggo'] == 'doggo') & (master_copy['pupper'] == 'pupper')
cond_3 = (master_copy['doggo'] == 'doggo') & (master_copy['puppo'] == 'puppo')
print(master_copy[cond_1 | cond_2 | cond_3][['tweet_id', 'text']])

                tweet_id  \
191   855851453814013952   
200   854010172552949760   
460   817777686764523521   
531   808106460588765185   
565   802265048156610565   
575   801115127852503040   
705   785639753186217984   
733   781308096455073793   
778   775898661951791106   
822   770093767776997377   
889   759793422261743616   
956   751583847268179968   
1063  741067306818797568   
1113  733109485275860992   

                                                                                                                                                                      text  
191   Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel        
200   At first I thought this was a shy doggo, but it's actually a Rare Canadian Floofer Owl. Amateurs would confuse the two. 11/10 only send dogs https://t.co/TXdT3tmuYk  
460   This is Dido. She's playing the lead role in "Pupper S

***Assign the following:***
1. 855851453814013952: puppo
2. 854010172552949760: floofer
3. 817777686764523521: pupper
4. 808106460588765185: pupper
5. 802265048156610565: pupper
6. 801115127852503040: pupper
7. 785639753186217984: pupper
8. 781308096455073793: pupper
9. 775898661951791106: pupper
10. 770093767776997377: pupper
11. 759793422261743616: pupper
12. 751583847268179968: doggo
13. 741067306818797568: doggo
14. 733109485275860992: doggo

**<span style="color: green">I like puppies, so for most of the entries it is pupper!</span>**

In [45]:
# assigning values.
master_copy.loc[master_copy['tweet_id'] == 855851453814013952, ['doggo', 'floofer', 'pupper']] = 'None'
master_copy.loc[master_copy['tweet_id'] == 854010172552949760, ['doggo', 'pupper', 'puppo']] = 'None'
master_copy.loc[master_copy['tweet_id'] == 817777686764523521, ['doggo', 'floofer', 'puppo']] = 'None'
master_copy.loc[master_copy['tweet_id'] == 808106460588765185, ['doggo', 'floofer', 'puppo']] = 'None'
master_copy.loc[master_copy['tweet_id'] == 802265048156610565, ['doggo', 'floofer', 'puppo']] = 'None'
master_copy.loc[master_copy['tweet_id'] == 801115127852503040, ['doggo', 'floofer', 'puppo']] = 'None'
master_copy.loc[master_copy['tweet_id'] == 785639753186217984, ['doggo', 'floofer', 'puppo']] = 'None'
master_copy.loc[master_copy['tweet_id'] == 781308096455073793, ['doggo', 'floofer', 'puppo']] = 'None'
master_copy.loc[master_copy['tweet_id'] == 775898661951791106, ['doggo', 'floofer', 'puppo']] = 'None'
master_copy.loc[master_copy['tweet_id'] == 770093767776997377, ['doggo', 'floofer', 'puppo']] = 'None'
master_copy.loc[master_copy['tweet_id'] == 759793422261743616, ['doggo', 'floofer', 'puppo']] = 'None'
master_copy.loc[master_copy['tweet_id'] == 751583847268179968, ['pupper', 'floofer', 'puppo']] = 'None'
master_copy.loc[master_copy['tweet_id'] == 741067306818797568, ['pupper', 'floofer', 'puppo']] = 'None'
master_copy.loc[master_copy['tweet_id'] == 733109485275860992, ['pupper', 'floofer', 'puppo']] = 'None'

####  Test - 1

In [46]:
# all values have been properly assigned
pd.set_option('display.max_colwidth', 50)
master_copy[cond_1 | cond_2 | cond_3][['tweet_id', 'doggo', 'floofer', 'pupper', 'puppo']]

,tweet_id,doggo,floofer,pupper,puppo
191,855851453814013952,None,None,None,puppo
200,854010172552949760,None,floofer,None,None
460,817777686764523521,None,None,pupper,None
531,808106460588765185,None,None,pupper,None
565,802265048156610565,None,None,pupper,None
575,801115127852503040,None,None,pupper,None
705,785639753186217984,None,None,pupper,None
733,781308096455073793,None,None,pupper,None
778,775898661951791106,None,None,pupper,None
822,770093767776997377,None,None,pupper,None


In [47]:
# selecting those, tweets that have no retweets
master_copy = master_copy[pd.isnull(master_copy['retweeted_status_id'])]

# deleting duplicates if any
master_copy = master_copy.drop_duplicates()

# deleting those tweets with no images.
master_copy = master_copy.dropna(subset = ['jpg_url'])

# reseting index
master_copy.reset_index(drop=True, inplace=True)

# droping columns
master_copy = master_copy.drop(labels = ['retweeted_status_id',
                                         'retweeted_status_user_id',
                                         'retweeted_status_timestamp',
                                         'in_reply_to_status_id', 
                                         'in_reply_to_user_id'],
                               axis = 1)

#### Test - 2

In [48]:
# after droping the columns, we should have about 25 dimensions/columns
master_copy.shape

(1685, 25)

#### 2. Condense wide-format to long-format

#### Define
- Condense `doggo`, `floofer`, `pupper`, `puppo` as `dog_class`.

#### Code

In [49]:
# to make sure that we have 
doggo = master_copy.doggo.value_counts()['doggo']
floofer = master_copy.floofer.value_counts()['floofer']
pupper = master_copy.pupper.value_counts()['pupper']
puppo = master_copy.puppo.value_counts()['puppo']

# printing count of each class
print("Count of Doggo: {}\nCount of Floofer: {}\nCount of Pupper: {}\nCount of Puppo: {}".format(doggo,
                                                                                                 floofer,
                                                                                                 pupper,
                                                                                                 puppo))

Count of Doggo: 57
Count of Floofer: 8
Count of Pupper: 173
Count of Puppo: 22


In [50]:
# selecting the columns that are to be melted
columns_to_melt = ['doggo', 'floofer', 'pupper', 'puppo']
columns_to_stay = [x for x in master_copy.columns.tolist() if x not in columns_to_melt]

# melting the the columns into values
master_copy = pd.melt(master_copy, id_vars = columns_to_stay, value_vars = columns_to_melt, 
                         var_name = 'stages', value_name = 'dog_class')

# Delete column 'stages'
master_copy = master_copy.drop('stages', 1)

# dropping duplicates
master_copy = master_copy.sort_values('dog_class').drop_duplicates('tweet_id', keep = 'last')
master_copy.reset_index(drop=True, inplace=True)

#### Test

In [51]:
# let's assert
assert doggo == master_copy.dog_class.value_counts()['doggo'], "Some entries are missing"
assert floofer == master_copy.dog_class.value_counts()['floofer'], "Some entries are missing"
assert pupper == master_copy.dog_class.value_counts()['pupper'], "Some entries are missing"
assert puppo == master_copy.dog_class.value_counts()['puppo'], "Some entries are missing"

#### 3. Fix all inaccurate data.

#### Define
- fix names of dogs.
- fix ratings.
- check source column.

#### Code

In [52]:
# Checking source column
master_copy.source.nunique()

3

**<span style="color: red">Okay! only three values, a categorical variable</span>**

In [53]:
import re
# assiging unique values to source.
master_copy['source'] = master_copy['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])

####  Test - 1

In [54]:
# taking a look at sample of 5 rows
master_copy.sample(5)[['tweet_id', 'source', 'text']]

,tweet_id,source,text
207,699370870310113280,Twitter for iPhone,Say hello to Calbert. He doesn't have enough l...
461,709449600415961088,Twitter for iPhone,Meet Karma. She's just a head. Lost body durin...
1497,886366144734445568,Twitter for iPhone,This is Roscoe. Another pupper fallen victim t...
1229,735635087207878657,Twitter for iPhone,This is Dakota. He hasn't grow into his skin y...
1013,786709082849828864,Twitter for iPhone,"This is Logan, the Chow who lived. He solemnly..."


In [55]:
# fixing names
non_names = master_copy.name.str.islower()
non_names = list(set(master_copy[non_names]['name'].tolist()))
flag = master_copy.name.str.len() == 1 & master_copy.name.str.isupper()
non_names.append(master_copy[flag][['tweet_id', 'name']]['name'].tolist()[0])

In [56]:
# replacing all garbage names with none, once done, we'll use the text field to extract names
for name in master_copy.name:
    if name in non_names:
        master_copy.loc[master_copy['name'] == name, ['name']] = 'None'

In [57]:
# checking if there are any non_names after the operation
assert len(master_copy[(master_copy.name.str.islower()) & (flag)]) == 0, "Check code"

***The following are patterns observed in `text` field, we shall use the :***
- This is [name] ..
- Meet [name] ..
- Say hello to [name] ..
- .. named [name] ..
- .. name is [name] ..

We will treat those cases to get the names from the text of the tweet

In [58]:
# extracting names using regular expression.
dog_names = []

# assigning patterns
pattern_1 = r'(T|t)his\sis\s([^.|,]*)'
pattern_2 = r'Meet\s([^.|,]*)'
pattern_3 = r'Say\shello\sto\s([^.|,]*)'
pattern_4 = r'name\sis\s([^.|,]*)'

# looping through text and extracting names
for text in master_copy['text']:
    # Start with 'This is '
    if re.search(pattern_1, text):
        # if our match has alternate name
        if "(" in re.search(pattern_1, text).group(2):
            dog_names.append(re.search(pattern_1, text).group(2).split()[0])
        # if our match has AKA in it
        elif "AKA" in re.search(pattern_1, text).group(2):
            dog_names.append(re.search(pattern_1, text).group(2).split()[0])
        # if our name has two dogs
        elif '&amp;' in re.search(pattern_1, text).group(2):
            temp = re.search(pattern_1, text).group(2).split()
            if len(temp) == 1:
                dog_names.append(temp[0])
            elif len(temp) == 3:
                dog_names.append(temp[0]+"|"+temp[-1])
            else:
                dog_names.append(temp[0]+"|"+temp[-2])
        elif 'named' in re.search(pattern_1, text).group(2):
            temp = re.search(pattern_1, text).group(2).split()
            dog_names.append(temp[-1])
        # just appending the name
        else:
            dog_names.append(re.search(pattern_1, text).group(2))
    
    # Start with 'Meet '
    elif re.search(pattern_2, text):
        # if our name has two dogs
        if '&amp;' in re.search(pattern_2, text).group(1):
            temp = re.search(pattern_2, text).group(1).split()
            if len(temp) == 1:
                dog_names.append(temp[0])
            elif len(temp) == 3:
                dog_names.append(temp[0]+"|"+temp[-1])
            else:
                dog_names.append(temp[0]+"|"+temp[-2])
        # if our name has alternatives
        elif '(' in re.search(pattern_2, text).group(1):
            dog_names.append(re.search(pattern_2, text).group(1).split()[0])
        # just appending the name
        else:
            dog_names.append(re.search(pattern_2, text).group(1))
    
    # Start with 'Say hello to '
    elif re.search(pattern_3, text):
        # if our match has alternate name
        if '(' in re.search(pattern_3, text).group(1):
            dog_names.append(re.search(pattern_3, text).group(1).split()[0])
        # if our name has two dogs
        elif '&amp;' in re.search(pattern_3, text).group(1):
            temp = re.search(pattern_3, text).group(1).split()
            if len(temp) == 1:
                dog_names.append(temp[0])
            elif len(temp) == 3:
                dog_names.append(temp[0]+"|"+temp[-1])
            else:
                dog_names.append(temp[0]+"|"+temp[-2])
        else:
            dog_names.append(re.search(pattern_3, text).group(1))    
    
    # contains 'name is'
    elif re.search(pattern_4, text):
        if len(re.search(pattern_4, text).group(1).split()) == 1:
            dog_names.append(re.search(pattern_4, text).group(1))
        else:
            temp = re.search(pattern_4, text).group(1).split()
            dog_names.append(temp[0])
        
    # No name specified or other style
    else:
        dog_names.append('None')

# adding this new set of names to our master_copy
master_copy['dog_names'] = dog_names

In [59]:
# new non names.
non_names = []
pattern_4 = r'^[a-z].*'
for name in master_copy['dog_names']:
    if re.search(pattern_4, name):
        master_copy.loc[master_copy['dog_names'] == name, ['dog_names']] = 'None'
        non_names.append(name)

In [60]:
# dog_names with and to be replaced with |
for name in master_copy['dog_names']:
    master_copy['dog_names'] = master_copy['dog_names'].str.replace(pat = r'\sand\s', repl = "|", regex = True)

In [61]:
# we need to replace two cells, with names 'Sadie|&amp;', 'Phillippe ...',
pd.set_option('display.max_colwidth', -1)
master_copy[(master_copy['dog_names'] == "Philippe from Soviet Russia") |
            (master_copy['dog_names'] == "Sadie|&amp;")][['tweet_id', 'text', 'dog_names']]

,tweet_id,text,dog_names
23,667495797102141441,This is Philippe from Soviet Russia. Commanding leader. Misplaced other boot. Hung flag himself. 9/10 charismatic af https://t.co/5NhPV8E45i,Philippe from Soviet Russia
160,693280720173801472,This is Sadie and her 2 pups Shebang &amp; Ruffalo. Sadie says single parenting is challenging but rewarding. All 10/10 https://t.co/UzbhwXcLne,Sadie|&amp;


In [62]:
# setting them to correct ones
# the following tweet_id we different from our regexs
master_copy.loc[master_copy['dog_names'] == "Philippe from Soviet Russia", ['dog_names']] = 'Phillippe'
master_copy.loc[master_copy['dog_names'] == "Sadie|&amp;", ['dog_names']] = 'Sadie|Shebang|Ruffalo'
master_copy.loc[master_copy['tweet_id'] == 667509364010450944, ['dog_names']] = 'Tickles'
master_copy.loc[master_copy['tweet_id'] == 667546741521195010, ['dog_names']] = 'George'
master_copy.loc[master_copy['tweet_id'] == 667073648344346624, ['dog_names']] = 'Dave'
master_copy.loc[master_copy['tweet_id'] == 667177989038297088, ['dog_names']] = 'Daryl'
master_copy.loc[master_copy['tweet_id'] == 666835007768551424, ['dog_names']] = 'Cupit|Prencer'
master_copy.loc[master_copy['tweet_id'] == 668221241640230912, ['dog_names']] = 'Bo|Smittens'
master_copy.loc[master_copy['tweet_id'] == 668268907921326080, ['dog_names']] = 'Guss'
master_copy.loc[master_copy['tweet_id'] == 666058600524156928, ['dog_names']] = 'Paul Rand'
master_copy.loc[master_copy['tweet_id'] == 692142790915014657, ['dog_names']] = 'Teddy'
master_copy.loc[master_copy['tweet_id'] == 684097758874210310, ['dog_names']] = 'Lupe'
master_copy.loc[master_copy['tweet_id'] == 709198395643068416, ['dog_names']] = 'Cletus|Jerome|Alejandro|Burp|Titson'
master_copy.loc[master_copy['tweet_id'] == 671743150407421952, ['dog_names']] = 'Jacob'
master_copy.loc[master_copy['tweet_id'] == 669037058363662336, ['dog_names']] = 'Pancho|Peaches'
master_copy.loc[master_copy['tweet_id'] == 669363888236994561, ['dog_names']] = 'Zeus'
master_copy.loc[master_copy['tweet_id'] == 813217897535406080, ['dog_names']] = 'Atlas'
master_copy.loc[master_copy['tweet_id'] == 856526610513747968, ['dog_names']] = 'Charlie'
master_copy.loc[master_copy['tweet_id'] == 861288531465048066, ['dog_names']] = 'Boomer'
master_copy.loc[master_copy['tweet_id'] == 863079547188785154, ['dog_names']] = 'Pipsy'
master_copy.loc[master_copy['tweet_id'] == 844979544864018432, ['dog_names']] = 'Toby'
master_copy.loc[master_copy['tweet_id'] == 836001077879255040, ['dog_names']] = 'Atlas'
master_copy.loc[master_copy['tweet_id'] == 758041019896193024, ['dog_names']] = 'Teagan'
master_copy.loc[master_copy['tweet_id'] == 765395769549590528, ['dog_names']] = 'Zoey'
master_copy.loc[master_copy['tweet_id'] == 778408200802557953, ['dog_names']] = 'Loki'
master_copy.loc[master_copy['tweet_id'] == 770069151037685760, ['dog_names']] = 'Carbon'

In [63]:
# dropping column name
master_copy = master_copy.drop(['name'], axis = 1)

# printing columns in master_copy
master_copy.columns.tolist()

['tweet_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'favorites',
 'retweets',
 'dog_class',
 'dog_names']

#### Test - 2

In [64]:
# selecting a dog_names that are none.
master_copy[master_copy['dog_names'] == 'None'].sample(5)[['tweet_id', 'text', 'dog_names']]

,tweet_id,text,dog_names
1163,747885874273214464,This is a mighty rare blue-tailed hammer sherk. Human almost lost a limb trying to take these. Be careful guys. 8/10 https://t.co/TGenMeXreW,None
1513,697616773278015490,This pupper doubles as a hallway rug. Very rare. Versatile af. 11/10 https://t.co/Jxd5pR02Cn,None
994,845812042753855489,We usually don't rate polar bears but this one seems extra good. Majestic as h*ck. 13/10 would hug for a while https://t.co/TLNexlqzXP,None
652,670717338665226240,*screams for a little bit and then crumples to the floor shaking* 12/10 https://t.co/W2MCt9pTed,None
366,718540630683709445,Get you a pup that can do both. 10/10 https://t.co/zSbyvm62xZ,None


####  Define
- Let's get cleaning the ratings.

#### Code

In [65]:
# as we are aware that there are two ratings in the text columns, lets use the
# regex to extract and replace the wrong ones.
ratings = master_copy['text'].apply(lambda x: re.findall(r'(\d+(\.\d+)|(\d+))\/(\d+0)', x))

In [66]:
# let's scale number from 0 to 15
def scale_rate(number, mini = 0, maxi = 15):
    return (number - mini)/(maxi - mini)

In [67]:
import math
# our temp variables
rate_num = []
rate_denom = []

# let's loop over and assign values properly
for rate in ratings:
    # if our regex didn't return any value.
    if len(rate) == 0:
        rate_num.append(0)
        rate_denom.append(0)
    
    # if regex's leght equals to one
    elif len(rate) == 1:
        
        temp = float(rate[0][0])
        
        # if we the value falls in between  [30,100]
        if 30 < temp < 100:
            temp_2 = int(rate[0][0]) / int(rate[0][-1][0])
            rate_num.append(math.ceil(temp_2))
            rate_denom.append(10)
        
        # else if our number falls between [100, 200]
        elif 100 < temp < 200:
            temp_2 = int(rate[0][0]) / int(rate[0][-1][:2])
            rate_num.append(math.ceil(temp_2))
            rate_denom.append(10)
        
        # else just ceiling the number
        else:
            rate_num.append(math.ceil(temp))
            rate_denom.append(10)
    
    # if our regex returned two ratings
    elif len(rate) == 2:
        
        # restricting our ratings to a max of 15
        if int(rate[0][0]) + int(rate[1][0]) > 15:
            temp = (int(rate[0][0]) + int(rate[1][0]))/2
            rate_num.append(math.ceil(temp))
            rate_denom.append(10)
        # if it is < 15
        else:
            rate_num.append(int(rate[0][0]) + int(rate[1][0]))
            rate_denom.append(10)
    
    # all others lenghts
    else:
        temp_sum = 0
        for i in range(len(rate)):
            temp_sum += int(rate[i][0])
        
        scaled = scale_rate(temp_sum)
        
        rate_num.append(math.ceil(scaled))
        rate_denom.append(10)

# assigning the values to rating_numerator and denominator
master_copy['rating_numerator'] = rate_num
master_copy['rating_denominator'] = rate_denom

#### Test

In [68]:
# lenght of the newly obtained values equal to the number of datapoint in our dataset
# i.e. 1686
print(len(master_copy['rating_numerator']), len(master_copy['rating_denominator']))

1685 1685


#### 4. Fix the structure of the table

#### Define
- Assign each column with appropriate type

#### Code

In [69]:
# info about master_copy
master_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1685 entries, 0 to 1684
Data columns (total 22 columns):
tweet_id              1685 non-null int64
timestamp             1685 non-null object
source                1685 non-null object
text                  1685 non-null object
expanded_urls         1685 non-null object
rating_numerator      1685 non-null int64
rating_denominator    1685 non-null int64
jpg_url               1685 non-null object
img_num               1685 non-null float64
p1                    1685 non-null object
p1_conf               1685 non-null float64
p1_dog                1685 non-null object
p2                    1685 non-null object
p2_conf               1685 non-null float64
p2_dog                1685 non-null object
p3                    1685 non-null object
p3_conf               1685 non-null float64
p3_dog                1685 non-null object
favorites             1685 non-null float64
retweets              1685 non-null float64
dog_class             1685 non

In [71]:
# assign each column with appropriate type
master_copy['tweet_id'] = master_copy['tweet_id'].astype(object)
master_copy['timestamp'] = pd.to_datetime(master_copy.timestamp)
master_copy['source'] = master_copy['source'].astype('category')
master_copy['favorites'] = master_copy['favorites'].astype(np.int64)
master_copy['retweets'] = master_copy['retweets'].astype(np.int64)
master_copy['dog_class'] = master_copy['dog_class'].astype('category')
master_copy['rating_numerator'] = master_copy['rating_numerator'].astype(np.int64)
master_copy['rating_denominator'] = master_copy['rating_denominator'].astype(np.int64)

#### Test

In [72]:
# printing information
master_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1685 entries, 0 to 1684
Data columns (total 22 columns):
tweet_id              1685 non-null object
timestamp             1685 non-null datetime64[ns]
source                1685 non-null category
text                  1685 non-null object
expanded_urls         1685 non-null object
rating_numerator      1685 non-null int64
rating_denominator    1685 non-null int64
jpg_url               1685 non-null object
img_num               1685 non-null float64
p1                    1685 non-null object
p1_conf               1685 non-null float64
p1_dog                1685 non-null object
p2                    1685 non-null object
p2_conf               1685 non-null float64
p2_dog                1685 non-null object
p3                    1685 non-null object
p3_conf               1685 non-null float64
p3_dog                1685 non-null object
favorites             1685 non-null int64
retweets              1685 non-null int64
dog_class             1

#### 5. Getting rid of predictions and add in final touches

#### Define
- Condense the `p_[1|2|3]` to `predicted_dog` and `conf_[1|2|3]` to `accuracy`.
- drop columns `img_num`, `p1`, `p1_conf`, `p1_dog`, `p2`, `p2_conf`, `p2_dog`, `p3`, `p3_conf`, `p3_dog`.
- rename columns that are apt to this dataset.

#### Code

In [73]:
# We will store the fisrt true algorithm
# with it's level of confidence
predicted_dog_breed = []
accuracy = []

# funvtion for getting the levels
def condense_predictions(dataframe):
    '''
    takes in the dataframe and extracts information for predicted dog breed.
    dataframe: input to the fuction
    '''
    if dataframe['p1_dog'] == True:
        predicted_dog_breed.append(dataframe['p1'])
        accuracy.append(dataframe['p1_conf'])
    elif dataframe['p2_dog'] == True:
        predicted_dog_breed.append(dataframe['p2'])
        accuracy.append(dataframe['p2_conf'])
    elif dataframe['p3_dog'] == True:
        predicted_dog_breed.append(dataframe['p3'])
        accuracy.append(dataframe['p3_conf'])
    else:
        predicted_dog_breed.append('NaN')
        accuracy.append(0)

master_copy.apply(condense_predictions, axis=1)
master_copy['dog_breeds'] = predicted_dog_breed
master_copy['accuracy'] = accuracy

In [75]:
# dropping columns
master_copy.drop(['img_num',
                  'p1', 'p1_conf', 'p1_dog',
                  'p2', 'p2_conf', 'p2_dog',
                  'p3', 'p3_conf', 'p3_dog'],
                 axis = 1,
                 inplace = True)

In [77]:
# final tocuh - renaming
master_copy.rename(columns={'source': 'tweet_source',
                            'text': 'tweet',
                            'expanded_urls' : 'tweet_urls',
                            'jpg_url': 'image_url',
                            'favorites': 'tweet_favorites',
                            'retweets': 'tweet_retweets'},
                   inplace = True)

####  Test

In [78]:
# printing information
master_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1685 entries, 0 to 1684
Data columns (total 14 columns):
tweet_id              1685 non-null object
timestamp             1685 non-null datetime64[ns]
tweet_source          1685 non-null category
tweet                 1685 non-null object
tweet_urls            1685 non-null object
rating_numerator      1685 non-null int64
rating_denominator    1685 non-null int64
image_url             1685 non-null object
tweet_favorites       1685 non-null int64
tweet_retweets        1685 non-null int64
dog_class             1685 non-null category
dog_names             1685 non-null object
dog_breeds            1685 non-null object
accuracy              1685 non-null float64
dtypes: category(2), datetime64[ns](1), float64(1), int64(4), object(6)
memory usage: 161.6+ KB


## Storing

**Done with the process of cleaning, let's store and analyse this dataset.**

In [79]:
# saving the file to local disk
master_copy.to_csv(folder+'/'+'twitter_archive_master.csv', index=False, encoding = 'utf-8')

# listing the dataset folder
os.listdir(folder)

['archive_copy.csv',
 'image-predictions.tsv',
 'master_set_raw.csv',
 'tweet_json.txt',
 'tweet_master.csv',
 'twitter-archive-enhanced.csv',
 'twitter_archive_master.csv']